In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
from IPython.display import Image
from keras.preprocessing import image
from keras import optimizers
from keras import layers,models
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
print(os.listdir("../input"))

import numpy as np


2025-08-18 20:30:55.758468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755549055.771421       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755549055.775469       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 20:30:55.791177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


['sample_submission.csv', 'description.md', 'test.zip', 'train.csv', 'sample_submission.csv.zip', 'test', 'train.zip', 'train', 'train.csv.zip', 'aerial-cactus-identification']


In [3]:
train_dir="../input/train/train"
test_dir="../input/test/test"
train=pd.read_csv('../input/train.csv')

df_test=pd.read_csv('../input/sample_submission.csv')


In [4]:
train.head(5)


,id,has_cactus
0,2de8f189f1dce439766637e75df0ee27.jpg,1
1,36704d250f236238e7f996812c48235d.jpg,1
2,eacde22fdc8c175972a5768e3daa8bc9.jpg,1
3,5d442f834da5e57d22b24802c32a8ca8.jpg,1
4,152491e0daf75c0e669400300ff7e645.jpg,1


In [5]:
print('out dataset has {} rows and {} columns'.format(train.shape[0],train.shape[1]))


out dataset has 14175 rows and 2 columns


In [6]:
train['has_cactus'].value_counts()


has_cactus
1    10628
0     3547
Name: count, dtype: int64

In [7]:
print("The number of rows in test set is %d"%(len(os.listdir('../input/test/test'))))


FileNotFoundError: [Errno 2] No such file or directory: '../input/test/test'

In [8]:
Image(os.path.join("../input/train/train",train.iloc[0,0]),width=250,height=250)


FileNotFoundError: No such file or directory: '../input/train/train/2de8f189f1dce439766637e75df0ee27.jpg'

FileNotFoundError: No such file or directory: '../input/train/train/2de8f189f1dce439766637e75df0ee27.jpg'

<IPython.core.display.Image object>

In [9]:
def prepare_data(data,m,direc):
    print('preparing data')
    X_train=np.zeros((m,150,150,3))
    count=0
    for fig in data['id']:
        img=image.load_img(os.path.join(direc,fig),target_size=(150,150,3))
        x=image.img_to_array(img)
        x=preprocess_input(x)
        X_train[count]=x/255
        count+=1
        
        
    print("Done")
    return X_train

        
        
    


In [10]:
data=prepare_data(train,train.shape[0],train_dir)
test=prepare_data(df_test,len(df_test),test_dir)


preparing data


FileNotFoundError: [Errno 2] No such file or directory: '../input/train/train/2de8f189f1dce439766637e75df0ee27.jpg'

In [11]:
X_train=data[:15001]
y_train=train['has_cactus'][:15001]
valid=data[15001:]
y_valid=train['has_cactus'][15001:]


NameError: name 'data' is not defined

In [12]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
         


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-18 20:31:01.685044: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [13]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.rmsprop(),metrics=['acc'])


AttributeError: module 'keras.api.optimizers' has no attribute 'rmsprop'

In [15]:
epochs=10
history=model.fit(X_train,y_train,epochs=10,validation_data=(valid,y_valid))


NameError: name 'X_train' is not defined

In [16]:
acc=history.history['acc']
epochs_=range(0,epochs)
plt.plot(epochs_,acc,label='training accuracy')

acc_val=history.history['val_acc']
plt.scatter(epochs_,acc_val,label="validation accuracy")

plt.legend()




    


NameError: name 'history' is not defined

In [17]:
acc=history.history['loss']
epochs_=range(0,epochs)
plt.plot(epochs_,acc,label='training loss')

acc_val=history.history['val_loss']
plt.scatter(epochs_,acc_val,label="validation loss")

plt.legend()


NameError: name 'history' is not defined

In [18]:
y_pre=model.predict_proba(test)


AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [19]:
df=pd.DataFrame({'id':df_test['id'] })
df['has_cactus']=y_pre
df.to_csv("submission.csv",index=False)


NameError: name 'y_pre' is not defined